In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import glob

import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data
import torchvision.transforms as T
from torchvision.models import resnet50
from torchsummary import summary

In [2]:
dataset_path = '/home/hari/data/Personal/flower_photos'
classes = glob.glob(os.path.join(dataset_path, '**'))
images = glob.glob(os.path.join(dataset_path, '**/*.jpg'))
print(len(classes))
print(len(images))

5
3670


In [3]:
for c in classes:
    images = glob.glob(os.path.join(dataset_path, c, '*.jpg'))
    print(len(images))

641
699
633
799
898


In [4]:
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224), antialias=False),
])

In [5]:
class Dataset(data.Dataset):
    def __init__(self, root_directory, transforms=None, train=True):
        self.root_directory = root_directory
        self.transforms = transforms
        self.train_size = 0.8
        self.train = train
        self.images = self.__train_validation_split__()
        classes = [c.split('/')[-1] for c in sorted(glob.glob(os.path.join(root_directory, '**')))]
        self.class_dict = {c : i for i, c in enumerate(classes)}

    def __train_validation_split__(self):
        images = glob.glob(os.path.join(self.root_directory, '**/*.jpg'))
        _size = int(len(images) * self.train_size) if self.train else int(len(images) * (1 - self.train_size))
        indices = np.random.choice(len(images), _size, replace=False)
        images = np.array(images)[indices]
        return images

    def __class_tonumeric__(self, label):
        return self.class_dict[label]
    
    def __getitem__(self, index):
        path = self.images[index]
        label = path.split('/')[-2]
        label = self.__class_tonumeric__(label)
        image = cv.imread(path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image)
        return image, label
    
    def __len__(self):
        return len(self.images)

In [6]:
dataset = Dataset(root_directory=dataset_path, transforms=transforms, train=True)
validset = Dataset(root_directory=dataset_path, transforms=transforms, train=False)
dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)
validloader = data.DataLoader(validset, batch_size=1, shuffle=True, drop_last=True)
X, y = next(iter(dataloader))
print(X.shape, y.shape)

torch.Size([4, 3, 224, 224]) torch.Size([4])


In [7]:
class LearnedReLU(nn.Module):
    def __init__(self, learning_epochs):
        super().__init__()
        self.learning_epochs = learning_epochs
        positive_coefficient = torch.normal(torch.ones(1), torch.ones(1))
        negative_coefficient = torch.normal(torch.zeros(1), torch.ones(1))
        self.positive_coefficient = nn.Parameter(positive_coefficient)
        self.negative_coefficient = nn.Parameter(negative_coefficient)

    def forward(self, X, epoch=None):
        _p_copy = self.positive_coefficient.data.clone()
        _n_copy = self.negative_coefficient.data.clone()
        if epoch is None or epoch >= self.learning_epochs:
            _p_copy, _n_copy = nn.Parameter(torch.ones(1)), nn.Parameter(torch.zeros(1))
        out = torch.where(X>=0, _p_copy * X, _n_copy * X)
        return out

class PerturbedReLU(nn.Module):
    def __init__(self, epsilon=0.01):
        super().__init__()
        self.epsilon = epsilon
    
    def forward(self, X):
        activated = F.relu(X)
        perturbation = torch.normal(torch.ones(X.shape)*self.epsilon, torch.ones(X.shape)*self.epsilon)
        return activated + perturbation

In [8]:
ctrl_model = resnet50(weights=None)
summary(ctrl_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [9]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, bias):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    
    def forward(self, X):
        return self.bn(self.conv(X))

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, num_instances, in_channels, out_channels, activation=nn.ReLU(inplace=True)):
        super().__init__()
        self.num_instances = num_instances
        self.conv1 = ConvBlock(in_channels=in_channels, out_channels=in_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.conv2 = ConvBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False)
        self.downsample = ConvBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False)
        self.activation = activation

    def forward(self, X):
        out = X.detach()
        out_down = out.detach().clone()
        for _ in range(self.num_instances):
            out = self.activation(self.conv1(out))
            out = self.conv2(out)
            out_down = self.downsample(out_down)
            out += out_down
        return out

In [11]:
class Resnet(nn.Module):
    def __init__(self, activation=nn.ReLU(inplace=True)):
        super().__init__()
        self.activation = activation
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.max_pool = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

        self.res_block_1 = ResidualBlock(in_channels=64, out_channels=256, activation=self.activation)
    
    def forward(self, X):
        out = self.bn1(self.conv1(X))
        out = self.activation(out)
        out = self.max_pool(out)
        out = self.res_block_1(out)
        out = self.activation(out)
        return out

In [12]:
model = Resnet()
out = model(X)
out.shape

torch.Size([4, 256, 56, 56])